## Explore scraping leftovers and figure out why they didn't scrape

There are about 60 certgovs where we didn't get results through the scrapy pipeline. This notebook looks at why and determines if there are new start urls we can try.

In [2]:
import numpy as np
import pandas as pd
import sqlite3

pd.set_option('display.max_rows', 75)
pd.set_option('display.max_columns', 100)

In [15]:
metadata_cols = [
        'referring_url', 
        'url',
        'base_domain',
        'file_type',
    ]
pdf_cols = [
        'url',
        'relative_filepath',
        'file_hash',
        'download_or_uptodate',
        'base_domain',
    ]

In [3]:
dbconn = sqlite3.connect("/data/data/webscraping/sbc_db_2022.sqlite")

In [4]:
leftover = pd.read_sql("SELECT * FROM gov_info WHERE is_scraped!=1;", dbconn)

In [5]:
leftover.shape

(33, 21)

In [6]:
leftover.head()

,ID,ID_CD,PLANT,SVY_YR,MNAME1,MNAME2,STREET,STREET2,PLACE,ST,ZIP9,id_idcd_plant,MNAME,start_url,external_domain,is_queried_search,date_queried_search,is_scraped,num_scraped,pdf_count,sbc_count
0,020000000,None,30700,None,CHATHAM REGIONAL EDUCATIONAL,ATTENDANCE AREA,None,None,None,AK,None,02000000030700,CHATHAM REGIONAL EDUCATIONAL ATTENDANCE AREA,https://gis.data.alaska.gov/search?tags=Govern...,None,1,2022-08-17 11:39:19.923085,0,0,0,0
1,051010010,None,30100,None,FRESNO COUNTY OFFICE OF EDUCATION,SPECIAL SCHOOL,None,None,None,CA,None,05101001030100,FRESNO COUNTY OFFICE OF EDUCATION SPECIAL SCHOOL,https://fcoe.org/office-education,None,1,2022-08-17 11:40:37.789660,0,0,0,0
2,051037037,None,00000,None,SAN DIEGO COUNTY,DEPT OF HUMAN RESOURCES/BENEFITS,None,None,None,CA,None,05103703700000,SAN DIEGO COUNTY DEPT OF HUMAN RESOURCES/BENEFITS,https://www.sandiegocounty.gov/content/sdc/hr/...,None,1,2022-08-17 11:41:22.208257,0,0,0,0
3,051043042,None,00000,None,SANTA CLARA COUNTY,EMPLOYEE SERVICES AGENCY,None,None,None,CA,None,05104304200000,SANTA CLARA COUNTY EMPLOYEE SERVICES AGENCY,https://employeeservices.sccgov.org/home,None,1,2022-08-17 14:00:19.987008,0,0,0,0
4,092001001,None,02300,None,NOMA BUSINESS IMPROVEMENT DISTRICT,,None,None,None,DC,None,09200100102300,NOMA BUSINESS IMPROVEMENT DISTRICT,https://www.nomabid.org/,None,1,2022-08-17 14:02:39.123449,0,0,0,0


Okay, first of all, how many of these URLs directly link to a pdf? If they don't link to a PDF, what are they like?

In [7]:
leftover['link_is_pdf'] = leftover['start_url'].str.endswith(".pdf")
leftover['link_is_pdf'].sum()

1

In [8]:
leftover.loc[~leftover['link_is_pdf']].head(10)

,ID,ID_CD,PLANT,SVY_YR,MNAME1,MNAME2,STREET,STREET2,PLACE,ST,ZIP9,id_idcd_plant,MNAME,start_url,external_domain,is_queried_search,date_queried_search,is_scraped,num_scraped,pdf_count,sbc_count,link_is_pdf
0,020000000,None,30700,None,CHATHAM REGIONAL EDUCATIONAL,ATTENDANCE AREA,None,None,None,AK,None,02000000030700,CHATHAM REGIONAL EDUCATIONAL ATTENDANCE AREA,https://gis.data.alaska.gov/search?tags=Govern...,None,1,2022-08-17 11:39:19.923085,0,0,0,0,False
1,051010010,None,30100,None,FRESNO COUNTY OFFICE OF EDUCATION,SPECIAL SCHOOL,None,None,None,CA,None,05101001030100,FRESNO COUNTY OFFICE OF EDUCATION SPECIAL SCHOOL,https://fcoe.org/office-education,None,1,2022-08-17 11:40:37.789660,0,0,0,0,False
2,051037037,None,00000,None,SAN DIEGO COUNTY,DEPT OF HUMAN RESOURCES/BENEFITS,None,None,None,CA,None,05103703700000,SAN DIEGO COUNTY DEPT OF HUMAN RESOURCES/BENEFITS,https://www.sandiegocounty.gov/content/sdc/hr/...,None,1,2022-08-17 11:41:22.208257,0,0,0,0,False
3,051043042,None,00000,None,SANTA CLARA COUNTY,EMPLOYEE SERVICES AGENCY,None,None,None,CA,None,05104304200000,SANTA CLARA COUNTY EMPLOYEE SERVICES AGENCY,https://employeeservices.sccgov.org/home,None,1,2022-08-17 14:00:19.987008,0,0,0,0,False
4,092001001,None,02300,None,NOMA BUSINESS IMPROVEMENT DISTRICT,,None,None,None,DC,None,09200100102300,NOMA BUSINESS IMPROVEMENT DISTRICT,https://www.nomabid.org/,None,1,2022-08-17 14:02:39.123449,0,0,0,0,False
5,100000000,None,00000,None,STATE OF FLORIDA,DIVISION OF STATE GROUP INSURANCE,None,None,None,FL,None,10000000000000,STATE OF FLORIDA DIVISION OF STATE GROUP INSUR...,https://www.dms.myflorida.com/workforce_operat...,None,1,2022-08-17 14:02:49.847583,0,0,0,0,False
6,105013001,None,00000,None,MIAMI-DADE COUNTY SCHOOL DISTRICT,,None,None,None,FL,None,10501300100000,MIAMI-DADE COUNTY SCHOOL DISTRICT,https://www.dadeschools.net/,None,1,2022-08-18 13:54:30.514133,0,0,0,0,False
7,115075001,None,00000,None,HENRY CO SCH DIST,,None,None,None,GA,None,11507500100000,HENRY CO SCH DIST,https://www.co.henry.ga.us/Departments/D-L/Hum...,None,1,2022-08-18 13:55:41.188369,0,0,0,0,False
8,122002001,None,00000,None,HONOLULU CITY AND COUNTY,,None,None,None,HI,None,12200200100000,HONOLULU CITY AND COUNTY,https://www.honolulu.gov/hr/benefits.html,None,1,2022-08-18 13:55:45.593683,0,0,0,0,False
9,150000000,None,00900,None,BALL STATE UNIVERSITY,,None,None,None,IN,None,15000000000900,BALL STATE UNIVERSITY,https://www.bsu.edu/about/administrativeoffice...,None,1,2022-08-18 13:56:10.521140,0,0,0,0,False


In [ ]:
# detective work - look through metadata to see if there's anything in there
metadata = pd.read_sql("SELECT * FROM latest_scrape;", dbconn)

In [23]:
metadata.shape

(1146942, 5)

In [28]:
metadata.loc[metadata['base_domain'].str.contains("honolulu")]

,referring_url,url,base_domain,file_type,id_idcd_plant


Are any of the start_urls that are PDFs SBC forms? Can we get better start urls for the ones that are PDFs?

In [33]:
leftover.loc[leftover['link_is_pdf']].start_url.to_list()

['http://marionmilitary.edu/core/uploads/2018/02/FACULTY-STAFF-HANDBOOK-feb-2018.pdf',
 'https://www.swrsd.org/site/handlers/filedownload.ashx?moduleinstanceid=236&dataid=773&FileName=20191125_AdminNegotiatedAgreementSIGNED.pdf',
 'https://www.akml.org/wp-content/uploads/2022/03/20220311-City-Administrator-job-description-final-draft.pdf',
 'https://chevakschool.org/wp-content/uploads/2019/12/CEA-NA-2018-2021.pdf',
 'https://aleutcorp.com/wp-content/uploads/2020/10/Service-Directory-2019.pdf',
 'https://www.phoenix.gov/hrsite/Documents/Benefits/2022-COPHX-Active-Guide%209-13-21.pdf',
 'https://www.mpsaz.org/benefits/files/benefits_and_enrollment_guide_2020-2021_-_final_1.pdf',
 'https://www.sblafco.org/files/6f3d6001f/Business_Item_No__5___Employee_Handbook_Addition.pdf',
 'https://ridelbt.com/wp-content/uploads/2019/01/FY-2018-Comprehensive-Annual-Financial-Report.pdf',
 'http://jhmbhealthconnect.com/wp-content/uploads/2012/10/Fush-2012_10-Plan-Booklet-FINAL-with-Inserts-I-III.pdf',
 

Note: I implemented a `parse_start_url` method in the scrapy spider and these are now handled in the same way as links to PDFs. As it turns out, none of these were SBC forms.

I tried re-scraping the remaining 30 or so links that looked good but didn't work the first time. Here's the metadata:

In [14]:
metadata = pd.read_csv("/data/data/webscraping/scraped_data/2022-09-23_1536_scrape_run_metadata.csv", names=metadata_cols)
metadata.head()

,referring_url,url,base_domain,file_type
0,NaN,https://gis.data.alaska.gov/search?tags=Govern...,gis.data.alaska.gov,b'text/html; charset=utf-8'
1,NaN,https://www.co.henry.ga.us/Departments/D-L/Hum...,www.co.henry.ga.us,b'text/html'
2,NaN,https://www.bmhahousing.com/DocumentCenter/Vie...,www.bmhahousing.com,b'application/pdf'
3,NaN,https://www3.dadeschools.net,www.dadeschools.net,b'text/html'
4,NaN,https://www.baltimorecityschools.org:443/retir...,www.baltimorecityschools.org,b'text/html; charset=utf-8'


In [17]:
pdf_metadata =  pd.read_csv("/data/data/webscraping/scraped_data/2022-09-23_1536_pdfs_from_sbc_spider.csv", names=pdf_cols)
pdf_metadata.head(20)

,url,relative_filepath,file_hash,download_or_uptodate,base_domain
0,https://www.bmhahousing.com/DocumentCenter/Vie...,full/2de69e799d239e548fad82e75484ba2b937a5173,a4b1220b22bd694205f4f09d1bd4b190,downloaded,www.bmhahousing.com
1,https://www.norfolkhealthcareconsortium.com/Do...,full/3a65a26f0b1a50128576cc77fe05d370a20e6367,374fd89db50a71fe331a17039e81bb53,downloaded,www.norfolkhealthcareconsortium.com
2,https://www.stgeorgefire.com/wp-content/upload...,full/cbe37d1c31443e8f325a4573c39ca717eb21e26a.pdf,5f7c68db5a58446997cfe9eff375895c,uptodate,www.stgeorgefire.com


In [19]:
pdf_metadata.url.to_list()

['https://www.bmhahousing.com/DocumentCenter/View/186/Assistant-Administrator-of-Personnel?bidId=',
 'https://www.norfolkhealthcareconsortium.com/DocumentCenter/View/913/2022-NRHA-Branded-Benefits-Guide-6-7-2022?bidId=',
 'https://www.stgeorgefire.com/wp-content/uploads/2022/09/BUDGET-Notice-2022.pdf']

In [21]:
pdf_metadata2 =  pd.read_csv("/data/data/webscraping/scraped_data/2022-09-23_1602_pdfs_from_sbc_spider.csv", names=pdf_cols)
pdf_metadata2.shape

(2039, 5)

In [22]:
pdf_metadata2.head(20)

,url,relative_filepath,file_hash,download_or_uptodate,base_domain
0,https://www.hca.wa.gov/assets/pebb/sebb-genera...,full/1cf44a2f4165245d3857aaf9b8d45f0fe0ee52be.pdf,7f19cbd7c3fe8b1302101b512f18f3fa,uptodate,www.hca.wa.gov
1,https://www.hca.wa.gov/assets/pebb/sebb-waivin...,full/09ae503957b9dced84ffbb2e1e05b221075e5d74.pdf,7725ce8956b6b1152890566adb30ae77,uptodate,www.hca.wa.gov
2,https://www.hca.wa.gov/assets/pebb/sebb-apple-...,full/a43038419f8490aaab80279fac05255585d54fea.pdf,1371bc82e28026fae2e3956d30612d9d,uptodate,www.hca.wa.gov
3,https://www.hca.wa.gov/assets/pebb/sebb-benefi...,full/268fc5a20b52b596f22b6e581a38e5bd6c48a11a.pdf,985ff54702490764ff15729eeb43859a,uptodate,www.hca.wa.gov
4,https://www.hca.wa.gov/assets/pebb/sebb-histor...,full/4cf78304fd42e7dadbb9991910de59fbc0dc6f24.pdf,ac3d7386df7fbe7ecf6f5167588b1ee4,uptodate,www.hca.wa.gov
5,https://www.hca.wa.gov/assets/pebb/k12-benefit...,full/51eee6a0dc9dadaede38430b469e1d337a67dfe9.pdf,dd622fe59580c6200c47619d89397fe5,uptodate,www.hca.wa.gov
6,https://www.hca.wa.gov/assets/program/county-c...,full/16c503c7b6c5c24f5dc0cdcdf8ea0fa839fb0f51.pdf,a7eb6a54c1ace1c22160956e4c00c009,uptodate,www.hca.wa.gov
7,https://www.hca.wa.gov/assets/free-or-low-cost...,full/0bfd88eb3567c2750f2b2bdf7a278b7ae76ba427.pdf,bce6f58284e43203b8c4b2ad2cd1f79e,uptodate,www.hca.wa.gov
8,https://www.hca.wa.gov/assets/free-or-low-cost...,full/fe1a4f489472826f4c801b86ea1603ccefd6523d.pdf,db36d8a8759e139bc40020e41b39a745,uptodate,www.hca.wa.gov
9,https://www.hca.wa.gov/assets/program/delivery...,full/5845696026941e96186999c069bff366fdf03439.pdf,1b098a5896410cfedd9302731f6b7658,uptodate,www.hca.wa.gov


### Can we use a lower tier url?

In [9]:
ids_for_pdfs = leftover.id_idcd_plant.to_list()

In [10]:
alt_start_links = pd.read_sql("SELECT * FROM google_query_results", dbconn)
alt_start_links = alt_start_links.loc[alt_start_links['id_idcd_plant'].isin(ids_for_pdfs)]
alt_start_links.shape

(330, 6)

In [11]:
alt_start_links.loc[alt_start_links['url_index']==2]

,url_index,start_url,id_idcd_plant,MNAME,is_queried_search,date_queried_search
491,2,https://education.alaska.gov/Media/Default/sta...,02000000030700,CHATHAM REGIONAL EDUCATIONAL ATTENDANCE AREA,1,2022-08-17 11:39:19.923085
921,2,https://www.fcoe.org/,05101001030100,FRESNO COUNTY OFFICE OF EDUCATION SPECIAL SCHOOL,1,2022-08-17 11:40:37.789660
1131,2,https://www.dhcs.ca.gov/,05103703700000,SAN DIEGO COUNTY DEPT OF HUMAN RESOURCES/BENEFITS,1,2022-08-17 11:41:22.208257
1251,2,https://home.sccgov.org/government/agencies-de...,05104304200000,SANTA CLARA COUNTY EMPLOYEE SERVICES AGENCY,1,2022-08-17 14:00:19.987008
1911,2,https://www.bizjournals.com/washington/news/20...,09200100102300,NOMA BUSINESS IMPROVEMENT DISTRICT,1,2022-08-17 14:02:39.123449
1961,2,https://www.mybenefits.myflorida.com/,10000000000000,STATE OF FLORIDA DIVISION OF STATE GROUP INSUR...,1,2022-08-17 14:02:49.847583
2361,2,https://www.miamidade.gov/global/humanresource...,10501300100000,MIAMI-DADE COUNTY SCHOOL DISTRICT,1,2022-08-18 13:54:30.514133
2691,2,https://schoolwires.henry.k12.ga.us/Page/115202,11507500100000,HENRY CO SCH DIST,1,2022-08-18 13:55:41.188369
2711,2,https://ers.ehawaii.gov/,12200200100000,HONOLULU CITY AND COUNTY,1,2022-08-18 13:55:45.593683
2821,2,https://commcenter.bsu.edu/message/employee-qu...,15000000000900,BALL STATE UNIVERSITY,1,2022-08-18 13:56:10.521140
